<a href="https://colab.research.google.com/github/mauriciodev/tec_forecast/blob/main/examples/tec_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Obtaining code and data

In [2]:
!git clone https://github.com/mauriciodev/tec_forecast.git
%cd tec_forecast

Cloning into 'tec_forecast'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 144 (delta 76), reused 113 (delta 48), pack-reused 0
Receiving objects: 100% (144/144), 1.84 MiB | 20.74 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/tec_forecast


The .npy files contain one year of GIM data.


In [3]:
!gdown 1Sm_PiVUIabaew_3Y7sT0NWBqu7xsdHvi
!tar -xvzf tec_forecast.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1Sm_PiVUIabaew_3Y7sT0NWBqu7xsdHvi
To: /content/tec_forecast/tec_forecast.tar.gz
100% 229M/229M [00:02<00:00, 81.4MB/s]
codg2014.npy
codg2015.npy
codg2019.npy
codg2020.npy


# Configuring experiments
The experiments are listed on config.csv. The networks and hyperparameters are set on that file. We are going to use pandas to edit it, so that we will only run one training cycle for each network.
To batch train and test a network, change the batch_train and batch_test columns.

## The configuration file

In [14]:
df=pd.read_csv('/content/tec_forecast/config.csv')
df

,comment,experiment_name,batch_train,batch_test,tested,compare,batch_size,num_epochs,filters,model,...,prediction_window,train_time_sampling,test_time_sampling,resample_rate,train_npy_dataset,test_npy_dataset,random_seed,loss,optimizer,best_of
0,None,Repeat_19-20,False,False,True,True,4,30,8,usePrevious,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
1,None,Repeat_14-15,False,False,True,True,4,30,8,usePrevious,...,12,1,2,12,codg2014.npy,codg2015.npy,1,mae,adam,1
2,This is not a model,c1pg,False,False,True,True,1,1,0,c1pg,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
3,This is not a model,c1pg14-15,False,False,True,True,1,1,0,c1pg,...,12,2,2,12,c1pg2015.npy,codg2015.npy,1,mae,adam,1
4,Very low memory consumption,ANN_NtoN_12_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
5,Very low memory consumption,ANN_NtoN_18_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
6,Very low memory consumption,ANN_NtoN_24_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
7,Very low memory consumption,ANN_NtoN_30_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
8,Very low memory consumption,ANN_NtoN_36_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
9,deeptec1-19-20,iConvLSTM,True,True,True,True,16,200,48,iConvLSTM,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1


# Changing the configuration with Pandas
We set the number of experiments on column "best_of" to 1 and turn off every experiment except "ANN_NtoN_48_2", which is a very lightweight neural network.

In [18]:
#changing
df['batch_train']='False'
df['batch_test']='False'
df['best_of']=1
df.loc[df['experiment_name']=='ANN_NtoN_36_2', 'batch_train'] = 'True'
df.loc[df['experiment_name']=='ANN_NtoN_36_2', 'batch_test'] = 'True'
#saving
df.to_csv('/content/tec_forecast/config.csv', index=False)
#showing
df

,comment,experiment_name,batch_train,batch_test,tested,compare,batch_size,num_epochs,filters,model,...,prediction_window,train_time_sampling,test_time_sampling,resample_rate,train_npy_dataset,test_npy_dataset,random_seed,loss,optimizer,best_of
0,None,Repeat_19-20,False,False,True,True,4,30,8,usePrevious,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
1,None,Repeat_14-15,False,False,True,True,4,30,8,usePrevious,...,12,1,2,12,codg2014.npy,codg2015.npy,1,mae,adam,1
2,This is not a model,c1pg,False,False,True,True,1,1,0,c1pg,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
3,This is not a model,c1pg14-15,False,False,True,True,1,1,0,c1pg,...,12,2,2,12,c1pg2015.npy,codg2015.npy,1,mae,adam,1
4,Very low memory consumption,ANN_NtoN_12_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
5,Very low memory consumption,ANN_NtoN_18_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
6,Very low memory consumption,ANN_NtoN_24_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
7,Very low memory consumption,ANN_NtoN_30_2,False,False,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
8,Very low memory consumption,ANN_NtoN_36_2,True,True,True,True,32,200,150,ANN,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1
9,deeptec1-19-20,iConvLSTM,False,False,True,True,16,200,48,iConvLSTM,...,12,2,2,12,codg2019.npy,codg2020.npy,1,mae,adam,1


# Running experiments


In [19]:
%cd /content/tec_forecast
!python batch_run.py

/content/tec_forecast
Beginning experiment ANN_NtoN_36_2
 - Log file: /content/tec_forecast/output/ANN_NtoN_36_2/batch_log.txt
 - Training.
 - Testing.
Model found. Setting as trained.
Plotting experiments comparison.
/content/tec_forecast/output/Repeat_19-20/results.py
/content/tec_forecast/output/Repeat_14-15/results.py
/content/tec_forecast/output/ANN_NtoN_36_2/results.py
         Network  parameters   MAE  RMSE    r2  rmse (1st)  max error(first)  \
2  ANN_NtoN_36_2       31356 1.119 1.616 0.942       1.616            28.255   
0   Repeat_19-20           0 1.024 1.616 0.942       1.824            27.900   
1   Repeat_14-15           0 2.856 4.341 0.928       4.512            63.500   

   rmse (last)  max error (last)  time (min)  memory  epochs  train_mae  \
2        1.807            23.446       1.452       0      81      1.054   
0        1.809            26.800       0.138       0      11      0.963   
1        4.424            62.200       0.156       0      11      3.419   



# Preparing the results
The plotresults.py script prepares the output/results.csv table, with an ordered list of the best results found.

In [5]:
%cd /content/tec_forecast
!python plotresults.py

/content/tec_forecast
/content/tec_forecast/output/Repeat_19-20/results.py
/content/tec_forecast/output/Repeat_14-15/results.py
        Network  parameters   MAE  RMSE    r2  rmse (1st)  max error(first)  \
0  Repeat_19-20           0 1.024 1.616 0.942       1.824            27.900   
1  Repeat_14-15           0 2.856 4.341 0.928       4.512            63.500   

   rmse (last)  max error (last)  time (min)  memory  epochs  train_mae  \
0        1.809            26.800       0.138       0      11      0.963   
1        4.424            62.200       0.156       0      11      3.419   

   train_rmse  val_mae  val_rmse  mae_mean  mae_std  rmse_mean  rmse_std  \
0       1.521    0.991     1.569     1.024    0.000      1.616     0.000   
1       5.077    3.487     5.132     2.856    0.000      4.341     0.000   

   r2_mean  r2_std  
0    0.942   0.000  
1    0.928   0.000  


# The results are stored in the output folder

In [20]:
%cd /content/tec_forecast/output
!ls

/content/tec_forecast/output
ANN_NtoN_12_2	       EF-ConvLSTM_3x3		   iConvLSTM
ANN_NtoN_18_2	       EF-ConvLSTM_3x3_14-15	   IconvLSTM_14-15
ANN_NtoN_24_2	       EF_ConvLSTM_48		   mae.pdf
ANN_NtoN_30_2	       EF_ConvLSTM_60		   max.pdf
ANN_NtoN_36_2	       EF-ConvLSTMv2_1x1	   REF-ConvLSTMv1_1x1
bar_plot.pdf	       EF-ConvLSTMv2_1x1_14-15	   REF-ConvLSTMv1_1x1_14-15
c1pg		       EF-ConvLSTMv2_3x3	   REF-ConvLSTMv1_3x3
c1pg14-15	       EF-ConvLSTMv2_3x3_14-15	   REF-ConvLSTMv1_3x3_14-15
ConvLSTM_Bi	       EF-ConvLSTMv2_3x3_14-15_12  REF-ConvLSTMv1_3x3_14-15_12
ConvLSTM_N-1	       EF-ConvLSTMv2_3x3_14-15_24  REF-ConvLSTMv1_3x3_14-15_24
EF_ConvLSTM_12	       EF-ConvLSTMv2_3x3_14-15_48  REF-ConvLSTMv1_3x3_14-15_48
EF-ConvLSTM_1x1        EF-ConvLSTMv2_3x3_14-15_60  Repeat_14-15
EF-ConvLSTM_1x1_14-15  EF-ConvLSTMv3		   Repeat_19-20
EF_ConvLSTM_24	       EF-ConvLSTMv3_14-15	   results.csv
EF_ConvLSTM_36	       errors.pdf		   rmse.pdf


In [21]:
results=pd.read_csv("/content/tec_forecast/output/results.csv")
results

,Unnamed: 0,Network,parameters,MAE,RMSE,r2,rmse (1st),max error(first),rmse (last),max error (last),...,train_mae,train_rmse,val_mae,val_rmse,mae_mean,mae_std,rmse_mean,rmse_std,r2_mean,r2_std
0,2,ANN_NtoN_36_2,31356,1.119,1.616,0.942,1.616,28.255,1.807,23.446,...,1.054,1.509,1.058,1.532,1.119,0.0,1.616,0.0,0.942,0.0
1,0,Repeat_19-20,0,1.024,1.616,0.942,1.824,27.900,1.809,26.800,...,0.963,1.521,0.991,1.569,1.024,0.0,1.616,0.0,0.942,0.0
2,1,Repeat_14-15,0,2.856,4.341,0.928,4.512,63.500,4.424,62.200,...,3.419,5.077,3.487,5.132,2.856,0.0,4.341,0.0,0.928,0.0
